In [ ]:
# If you already have rasterio, imageio, etc. installed in the conda env you selected, skip the pip installs.
# If not installed, uncomment the pip lines and run them once.

# !pip install rasterio imageio matplotlib numpy

import os
import numpy as np
import rasterio
from rasterio.enums import Resampling
import imageio
import matplotlib.pyplot as plt
from pathlib import Path


In [ ]:
# ------------------- CHANGE THESE PATHS -------------------
# Provide the exact paths to the 3 Sentinel-2 resampled GeoTIFFs (L2A) you want to process.
# NOTE: these resampled files should contain bands in this order (as in your SNAP graph): B1, B2, B3, B4, B12
# If your files include other bands or a different order, see note below and adjust band indices.

s2_files = [
    r"D:\blatten_project\data\blatten_phase1\derived\s2_outputs\Subset_S2C_MSIL2A_20250408T103051_N0511_R108_T32TMS_20250408T161415_resampled.tif",
    r"D:\blatten_project\data\blatten_phase1\derived\s2_outputs\Subset_S2A_MSIL2A_20250430T102701_N0511_R108_T32TMS_20250430T190517_resampled.tif",
    r"D:\blatten_project\data\blatten_phase1\derived\s2_outputs\Subset_S2C_MSIL2A_20250518T102621_N0511_R108_T32TMS_20250518T143414_resampled.tif",
    r"D:\blatten_project\data\blatten_phase1\derived\s2_outputs\Subset_S2A_MSIL2A_20250530T103041_N0511_R108_T32TMS_20250530T142711_resampled.tif"
]

# Output folder for PNGs and GIF
OUT_DIR = Path(r"D:\blatten_project\data\blatten_phase1\derivedassets/s2_animation")
OUT_DIR.mkdir(parents=True, exist_ok=True)

# Target max dimension for output PNG (keeps files manageable). Set to 2048 for good quality.
TARGET_MAX_DIM = 2048

# Cloud threshold parameters
CLOUD_REFLECTANCE_THRESHOLD = 0.7   # reflectance >0.70 considered cloud pixel (after scaling to 0-1)
MAX_CLOUD_FRACTION = 0.55           # skip image if > 55% cloud as requested

# Color recipe scales (as you specified)
R_MIN, R_MAX = 0.20, 1.0   # for B12 -> R channel scale from 0.2..1 -> 0..1
G_MIN, G_MAX = 0.0, 1.0    # for B2  -> G channel scale 0..1
B_MIN, B_MAX = 0.0, 1.0    # for B1  -> B channel scale 0..1
# ---------------------------------------------------------


In [ ]:
def read_band_resample(path, band_index, max_dim=TARGET_MAX_DIM):
    """
    Read a single band from a GeoTIFF and optionally resample if too large.
    band_index: 1-based band index for rasterio.read()
    returns: ndarray (float32), transform, crs
    """
    with rasterio.open(path) as src:
        w, h = src.width, src.height
        # compute scale factor so max dimension <= max_dim
        scale = max(1.0, max(w / max_dim, h / max_dim))
        out_w = int(w / scale)
        out_h = int(h / scale)
        arr = src.read(band_index, out_shape=(out_h, out_w), resampling=Resampling.bilinear).astype('float32')
        # read transform and crs for possible later use (not strictly needed for PNG)
        # compute new transform if resampled:
        transform = src.transform * src.transform.scale((src.width / out_w), (src.height / out_h))
        crs = src.crs
    return arr, transform, crs

def detect_and_scale_reflectance(band):
    """
    Sentinel-2 L2A may be in [0..1] OR scaled [0..10000]. Detect and scale to [0..1].
    """
    if np.nanmax(band) > 1.5:
        # probably scaled by 10000
        band = band / 10000.0
    # clamp to valid reflectance range
    band = np.clip(band, 0.0, 1.0)
    return band

def channel_scale(band, lo, hi):
    """
    scale band values such that values <= lo -> 0, >= hi ->1. Linear in between.
    lo,hi expected in 0..1
    """
    # avoid division by zero
    if hi <= lo:
        raise ValueError("hi must be > lo for scaling")
    scaled = (band - lo) / (hi - lo)
    scaled = np.clip(scaled, 0.0, 1.0)
    return scaled


In [ ]:
processed_pngs = []
skipped_images = []

for fp in s2_files:
    fp = Path(fp)
    if not fp.exists():
        print(f"ERROR: file not found: {fp}")
        continue

    print("\nProcessing:", fp.name)

    # --- READ BANDS (adjust band indices here if your TIFF has different band order) ---
    # Assumption: band order is: B1 (index 1), B2 (index 2), B3 (3), B4 (4), B12 (5)
    # If your resampled TIFF ordering differs, change indices accordingly.
    try:
        b1, t, crs = read_band_resample(str(fp), 1)   # B1 -> Blue
        b2, _, _   = read_band_resample(str(fp), 2)   # B2 -> Green
        # optional: you could read other bands but only need B12 as red
        b12, _, _  = read_band_resample(str(fp), 5)   # B12 -> ShortWave-IR
    except Exception as e:
        print("Error reading bands. Check band indices and file structure.", e)
        raise

    # --- SCALE reflectance to 0..1 if needed ---
    b1 = detect_and_scale_reflectance(b1)
    b2 = detect_and_scale_reflectance(b2)
    b12 = detect_and_scale_reflectance(b12)

    # --- CLOUD FRACTION ESTIMATE (simple heuristic using Green band reflectance) ---
    cloud_mask = (b2 >= CLOUD_REFLECTANCE_THRESHOLD)
    cloud_frac = float(np.nanmean(cloud_mask))
    print(f"Estimated cloud fraction (by B2 >= {CLOUD_REFLECTANCE_THRESHOLD}): {cloud_frac:.3f}")

    if cloud_frac > MAX_CLOUD_FRACTION:
        print(f"SKIP image {fp.name} — cloud fraction {cloud_frac:.2%} > {MAX_CLOUD_FRACTION:.2%}")
        skipped_images.append((fp.name, cloud_frac))
        continue  # skip this file as per requirement

    # --- APPLY COLOR RECIPE ---
    # R = scaled B12 between R_MIN..R_MAX
    R = channel_scale(b12, R_MIN, R_MAX)
    G = channel_scale(b2,  G_MIN, G_MAX)
    B = channel_scale(b1,  B_MIN, B_MAX)

    # Optional: mild gamma correction for better visibility
    gamma = 1.0  # set to 1.0 for no gamma change. change to 0.9 or 0.8 if you want brighter midtones
    R = np.power(R, gamma)
    G = np.power(G, gamma)
    B = np.power(B, gamma)

    # stack to RGB
    rgb = np.dstack([R, G, B])
    # convert to uint8
    rgb_u8 = (np.clip(rgb, 0.0, 1.0) * 255).astype('uint8')

    # --- Save PNG ---
    out_png = OUT_DIR / (fp.stem + "_B12B2B1_rgb.png")
    imageio.imwrite(str(out_png), rgb_u8)
    print("Saved PNG:", out_png)
    processed_pngs.append(str(out_png))

print("\nDone. Processed:", len(processed_pngs), "PNG(s). Skipped:", len(skipped_images))
if skipped_images:
    print("Skipped details:", skipped_images)


In [ ]:
# Create GIF (optional). Only include the processed PNGs (skipped ones are ignored).
if len(processed_pngs) >= 1:
    frames = []
    for p in processed_pngs:
        frames.append(imageio.imread(p))
    gif_out = OUT_DIR / "s2_animation.gif"
    # duration: seconds per frame
    imageio.mimsave(str(gif_out), frames, duration=1000.0)
    print("Saved GIF:", gif_out)
else:
    print("No PNGs available to make GIF.")
